<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Leads-data" data-toc-modified-id="Leads-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Leads data</a></span></li><li><span><a href="#Q&amp;As-data" data-toc-modified-id="Q&amp;As-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Q&amp;As data</a></span></li><li><span><a href="#Join-dataframes" data-toc-modified-id="Join-dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Join dataframes</a></span></li></ul></div>

In [15]:
import pandas as pd

# Leads data

In [16]:
# import leads data

df_leads = pd.read_csv('data_files/scorer_ga_leads_by_cid.csv', dtype=object)
df_leads['key'] = df_leads['ga:date'] + '-' + df_leads['ga:dimension1']
df_leads

,ga:date,ga:dimension11,ga:dimension1,ga:uniqueEvents,key
0,20210707,162559549757326,1796177990.1625595499,1,20210707-1796177990.1625595499
1,20210707,162559815703238,1680934699.1625598157,1,20210707-1680934699.1625598157
2,20210707,162562667992520,1990645011.1625626682,1,20210707-1990645011.1625626682
3,20210707,162562677924885,1922511147.1625626779,1,20210707-1922511147.1625626779
4,20210707,162562682526032,2822540.1614454503,1,20210707-2822540.1614454503
...,...,...,...,...,...
78306,20211231,164100334152529,1808261425.1641003327,1,20211231-1808261425.1641003327
78307,20211231,164100403725164,1208938549.1600335399,1,20211231-1208938549.1600335399
78308,20211231,164100411161519,1498184985.1641004110,1,20211231-1498184985.1641004110
78309,20211231,164100425750682,1188319175.1629584584,1,20211231-1188319175.1629584584


# Q&As data

In [17]:
# import qas data only once

df_qa = pd.read_csv('data_files/scorer_ga_qas_by_cid.csv', dtype=object)
temp = df_qa.copy()

In [18]:
# preprocessing for q&a

# remove first row
temp = df_qa.iloc[1:]


# creates question & answer columns from event_action & evenet_label. This mixes home and business quoters
temp.loc[:, 'question'] = temp['ga:eventAction'].str.split('|').str[0]
temp.loc[:, 'answer'] = temp['ga:eventLabel'].str.split('|').str[1]


# creates key from date and cid
temp.loc[:, 'key'] = temp['ga:date'] + '-' + temp['ga:dimension1']


# remove useless
temp.drop(columns=['ga:eventCategory', 'ga:eventAction', 'ga:eventLabel', 'ga:segment', 'ga:uniqueEvents',
                     'ga:date', 'ga:dimension1'], inplace=True)

# creates pivot table with last record occurrence
df_qas = temp.pivot_table(index='key', columns='question', values='answer', aggfunc='last').reset_index()


# restablishes date, cid and key on pivot table
df_qas.loc[:, 'date'] = df_qas['key'].str.split('-').str[0]
df_qas.loc[:, 'cid'] = df_qas['key'].str.split('-').str[1]
df_qas


/opt/tljh/user/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/tljh/user/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/opt/tljh/user/lib/python3.6/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

question,key,0,step1,step10,step2,step3,step4,step5,step6,step7,step8,step9,date,cid
0,20210707-1000653101.1625702265,home,chalet,NaN,primary,insideUrban,streetLevel,garden,yes,yes,more4hours,yes,20210707,1000653101.1625702265
1,20210707-1004499971.1625700770,home,chalet,no,primary,insideUrban,streetLevel,balconyOrTerrace,yes,yes,2_4hours,no,20210707,1004499971.1625700770
2,20210707-1012284362.1625630376,home,chalet,NaN,primary,insideUrban,streetLevel,garden,no,no,2_4hours,no,20210707,1012284362.1625630376
3,20210707-1015111516.1625664236,home,chalet,NaN,primary,insideUrban,streetLevel,no,yes,yes,more4hours,no,20210707,1015111516.1625664236
4,20210707-1021680491.1625709500,home,chalet,no,primary,insideUrban,streetLevel,balconyOrTerrace,yes,yes,always,no,20210707,1021680491.1625709500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88804,20211231-990637517.1640952377,home,flat,no,primary,insideUrban,guardedUrbanization,garden,no,no,more4hours,no,20211231,990637517.1640952377
88805,20211231-993345027.1640953692,home,flat,no,primary,insideUrban,noGuardedUrbanization,balconyOrTerrace,no,no,more4hours,no,20211231,993345027.1640953692
88806,20211231-994837841.1640789122,home,chalet,NaN,primary,insideUrban,streetLevel,balconyOrTerrace,yes,yes,always,no,20211231,994837841.1640789122
88807,20211231-995985076.1635985051,home,flat,yes,primary,insideUrban,streetLevel,no,yes,yes,more4hours,yes,20211231,995985076.1635985051


# Join dataframes

In [19]:
df_ga = df_leads.merge(df_qas, on='key', how='left')

In [20]:
# check merge

df_ga = df_ga[['ga:date', 'date', 'ga:dimension1', 'cid', 'key', 'ga:dimension11', 'ga:uniqueEvents',
              '0', 'step1', 'step2', 'step3', 'step4', 'step5', 'step6', 'step7', 'step8', 'step9', 'step10']]

df_ga

,ga:date,date,ga:dimension1,cid,key,ga:dimension11,ga:uniqueEvents,0,step1,step2,step3,step4,step5,step6,step7,step8,step9,step10
0,20210707,20210707,1796177990.1625595499,1796177990.1625595499,20210707-1796177990.1625595499,162559549757326,1,business,warehouseIndustrialState,insideUrban,onlyMe,mornings,lowValue,lowValue,no,no,NaN,NaN
1,20210707,20210707,1680934699.1625598157,1680934699.1625598157,20210707-1680934699.1625598157,162559815703238,1,home,chalet,primary,insideUrban,streetLevel,garden,no,no,more4hours,no,no
2,20210707,20210707,1990645011.1625626682,1990645011.1625626682,20210707-1990645011.1625626682,162562667992520,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
3,20210707,20210707,1922511147.1625626779,1922511147.1625626779,20210707-1922511147.1625626779,162562677924885,1,NaN,NaN,NaN,NaN,streetLevel,balconyOrTerrace,no,no,less2hours,no,no
4,20210707,20210707,2822540.1614454503,2822540.1614454503,20210707-2822540.1614454503,162562682526032,1,home,flat,primary,insideUrban,noGuardedUrbanization,balconyOrTerrace,no,no,2_4hours,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78306,20211231,20211231,1808261425.1641003327,1808261425.1641003327,20211231-1808261425.1641003327,164100334152529,1,home,chalet,primary,insideUrban,streetLevel,balconyOrTerrace,no,no,always,yes,yes
78307,20211231,20211231,1208938549.1600335399,1208938549.1600335399,20211231-1208938549.1600335399,164100403725164,1,home,chalet,primary,insideUrban,guardedUrbanization,balconyOrTerrace,yes,yes,always,no,no
78308,20211231,20211231,1498184985.1641004110,1498184985.1641004110,20211231-1498184985.1641004110,164100411161519,1,home,chalet,primary,insideUrban,streetLevel,no,yes,yes,always,no,no
78309,20211231,20211231,1188319175.1629584584,1188319175.1629584584,20211231-1188319175.1629584584,164100425750682,1,home,chalet,primary,insideUrban,streetLevel,garden,yes,yes,2_4hours,no,no


In [21]:
# refine columns

df_ga.rename(columns={'0':'step0', 'ga:dimension11': 'ga_leadid'}, inplace=True)
df_ga.drop(columns=['ga:date', 'ga:dimension1', 'key', 'ga:uniqueEvents', 'date', 'cid'], inplace=True)
df_ga

,ga_leadid,step0,step1,step2,step3,step4,step5,step6,step7,step8,step9,step10
0,162559549757326,business,warehouseIndustrialState,insideUrban,onlyMe,mornings,lowValue,lowValue,no,no,NaN,NaN
1,162559815703238,home,chalet,primary,insideUrban,streetLevel,garden,no,no,more4hours,no,no
2,162562667992520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
3,162562677924885,NaN,NaN,NaN,NaN,streetLevel,balconyOrTerrace,no,no,less2hours,no,no
4,162562682526032,home,flat,primary,insideUrban,noGuardedUrbanization,balconyOrTerrace,no,no,2_4hours,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
78306,164100334152529,home,chalet,primary,insideUrban,streetLevel,balconyOrTerrace,no,no,always,yes,yes
78307,164100403725164,home,chalet,primary,insideUrban,guardedUrbanization,balconyOrTerrace,yes,yes,always,no,no
78308,164100411161519,home,chalet,primary,insideUrban,streetLevel,no,yes,yes,always,no,no
78309,164100425750682,home,chalet,primary,insideUrban,streetLevel,garden,yes,yes,2_4hours,no,no


In [22]:
# quick check

df_ga.isnull().sum() / len(df_ga) *100

ga_leadid     0.000000
step0        10.095644
step1        10.104583
step2        10.122460
step3        10.128845
step4        10.096921
step5        10.068828
step6        10.073936
step7        10.057335
step8        10.085429
step9        18.973069
step10       18.988392
dtype: float64

In [23]:
df_ga.dtypes

ga_leadid    object
step0        object
step1        object
step2        object
step3        object
step4        object
step5        object
step6        object
step7        object
step8        object
step9        object
step10       object
dtype: object

In [24]:
df_ga.to_csv('data_files/scorer_ga_mergedData.csv', index=False)